<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/sydney_rain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [82]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [83]:
!wget -O Weather_Data.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv

--2024-07-09 08:00:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284201 (278K) [text/csv]
Saving to: ‘Weather_Data.csv’

Weather_Data.csv    100%[===================>] 277.54K   353KB/s    in 0.8s    

2024-07-09 08:00:54 (353 KB/s) - ‘Weather_Data.csv’ saved [284201/284201]



In [84]:
df = pd.read_csv("Weather_Data.csv")
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


In [85]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [86]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

In [87]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

In [88]:

df_sydney_processed = df_sydney_processed.astype(float)

In [89]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

In [90]:
df2 = df_sydney_processed.to_numpy()

In [91]:
Y = Y.to_numpy()

In [92]:
X_train,X_test,y_train,y_test = train_test_split(df2,Y,train_size = 0.8,random_state = 0)

In [93]:
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

In [94]:
X_train.shape

torch.Size([2616, 67])

In [95]:
class neural(nn.Module):
  def __init__(self, in_size):
    super().__init__()
    self.output = nn.Sequential(
        nn.Linear(in_size, 10),
        nn.ReLU(),
        nn.Linear(10,1),
        nn.Sigmoid()
    )
  def forward(self, X):
    return self.output(X)


In [96]:
model = neural(in_size = 67).to(device)

In [97]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [98]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [99]:
torch.manual_seed(42)
epochs = 10000

In [100]:
for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model(X_train).squeeze()
    y_pred = torch.round(y_logits) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model(X_test).squeeze()
      test_pred = torch.round(test_logits) # logits -> prediction probabilities -> prediction labels
      # 2. Calcuate loss and accuracy
      test_loss = loss_fn(test_logits, y_test)
      test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 100 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 200 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 300 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 400 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 500 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 600 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 700 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 800 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 900 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 1000 | Loss: 0.69315, Accuracy: 74.46% | Test Loss: 0.69315, Test Accuracy: 72.37%
Epoch: 1100 | Loss: 0.